# Basically a wrong answer
Not the reverse since it takes as an input the indexes of zeros ... which would come from the string.

In [1]:
from typing import List, Optional, Tuple
from itertools import product

# ---------- helpers ----------

def compute_c_from_zeros(zeros: List[int]) -> int:
    """Compute c = sum_{j=0}^{B-1} 3^{B-1-j} * 2^{zeros[j]} for sorted zeros list."""
    if not zeros:
        return 0
    B = len(zeros)
    return sum((3 ** (B - j - 1)) * (1 << zeros[j]) for j in range(B))

def value_from_zeros(a: int, zeros: List[int]) -> Optional[int]:
    """
    Given length a and sorted zero indices, return the integer value (2^a - c)/3^B
    if divisible, otherwise return None.
    """
    B = len(zeros)
    if B == 0:
        # value is (2^a - 0)/1 = 2^a
        return 2**a
    c = compute_c_from_zeros(zeros)
    num = (1 << a) - c
    denom = 3 ** B
    if num >= 0 and num % denom == 0:
        return num // denom
    return None

# ---------- find canonical s_list for d ----------
def find_s_list_for_d(d: int, m_max: int = 5, s_max: int = 8) -> Optional[List[int]]:
    """
    Find a short canonical list s_list = [s_0, ..., s_{m-1}] such that
      sum_{r=0}^{m-1} 3^{m-1-r} * 2^{s_r} == d.
    We search m from 0..m_max and s_r in range(0..s_max).
    Returns the first solution found (small m, lexicographically small s_list), or None.
    """
    if d == 0:
        return []
    # loop small lengths m
    for m in range(1, m_max + 1):
        # iterate non-decreasing s_r? we allow any s_r (they will be placed as most-significant)
        # but to canonicalize, search s_r in nondecreasing order (so lexicographically canonical).
        # We'll generate combinations with repetition of s_r from 0..s_max.
        # Use product and then require sorted nondecreasing.
        for candidate in product(range(s_max + 1), repeat=m):
            if list(candidate) != sorted(candidate):
                continue
            total = 0
            for r, s in enumerate(candidate):
                total += (3 ** (m - 1 - r)) * (1 << s)
            if total == d:
                return list(candidate)
    return None

# ---------- main canonical T-insertion function ----------

def canonical_T_insertion(a: int, zeros: List[int], d: int,
                          T_start: Optional[int] = None, T_max: int = 2000,
                          m_max: int = 6, s_max: int = 20
                          ) -> Optional[Tuple[int, List[int], int]]:
    """
    Canonical lemma T-insertion construction.

    Inputs:
      - a: original bitstring length (int)
      - zeros: sorted list of zero indices for original string (z_0 < ... < z_{b-1})
      - d: digit to add (e.g. 1 for 4n+1)
      - T_start: optional starting T; if None, uses max(a+2, last_zero+2)+1
      - T_max: upper bound for T (search limit)
      - m_max, s_max: search bounds for canonical s_list for d

    Returns (a_new, final_zeros, c_new) on success where:
      - a_new = a + 2 (per lemma),
      - final_zeros: sorted zeros after uniform downshift (these correspond to the new bitstring),
      - c_new = compute_c_from_zeros(final_zeros).
    Returns None if no success within search bounds.
    """

    b = len(zeros)
    # Safety check: input zeros must be sorted and valid
    if any(zeros[i] >= zeros[i+1] for i in range(len(zeros)-1)):
        raise ValueError("zeros must be strictly increasing")

    # canonical s_list for d
    s_list = find_s_list_for_d(d, m_max=m_max, s_max=s_max)
    if s_list is None:
        # can't represent d in searched parameter range
        return None
    m = len(s_list)

    # compute n (the integer represented by current triple)
    c_old = compute_c_from_zeros(zeros)
    if (1 << a) - c_old < 0 or ((1 << a) - c_old) % (3 ** b) != 0:
        raise ValueError("Input (a,zeros) not consistent: (2^a - c) not divisible by 3^b")
    n = ((1 << a) - c_old) // (3 ** b)
    target = 4 * n + d

    # choose T_start if not provided
    last_old = zeros[-1] if zeros else -1
    minT = max(a + 2, last_old + 2) + 1
    if T_start is None:
        T = minT
    else:
        T = max(T_start, minT)

    a_new = a + 2

    # Try increasing T until success or until T_max
    while T <= T_max:
        # new most significant zeros positions: T + s for s in s_list
        new_ms = [T + s for s in s_list]
        # shifted old zeros by +2+T
        tilde = [z + 2 + T for z in zeros]
        zeros_combined = sorted(new_ms + tilde)

        # uniform downshift by T
        final_zeros = [z - T for z in zeros_combined]

        # compute the integer value produced by final_zeros with length a_new
        val = value_from_zeros(a_new, final_zeros)
        if val == target:
            # success: return new triple
            c_new = compute_c_from_zeros(final_zeros)
            return a_new, final_zeros, c_new

        # If not equal, increment T and retry
        T += 1

    # failed to find a T <= T_max
    return None


In [2]:
def Z(s):
    for i in range(len(s)):
        if s[i] == "0":
            yield i
#
